# Example of usage ImageAdaptiveThresholding and  ImageAdaptiveBinarizer transformers

## Install spark-ocr python packge
Need specify path to `spark-ocr-assembly-[version].jar` or `secret`

In [1]:
secret = ""
license = ""
version = secret.split("-")[0]
spark_ocr_jar_path = "../../target/scala-2.11"

In [ ]:
# install from PYPI using secret
%pip install spark-ocr==$version+spark32 --extra-index-url=https://pypi.johnsnowlabs.com/$secret --upgrade

In [ ]:
%pip install spark-nlp==3.4.0

In [4]:
#or install from local path
#%pip install ../../python/dist/spark-ocr-3.9.0+spark30.tar.gz

## Initialization of spark session

In [ ]:
from sparkocr import info
info()

In [ ]:
from pyspark.sql import SparkSession
from sparkocr import start
import sys
import os

if license:
    os.environ['JSL_OCR_LICENSE'] = license

spark = start(secret=secret, jar_path=spark_ocr_jar_path)
spark

## Import OCR transformers

In [ ]:
from pyspark.ml import PipelineModel

from sparkocr.transformers import *
from sparkocr.enums import *
from sparkocr.utils import display_image

## Read PDF document as binary file

In [ ]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-ocr-workshop/master/jupyter/data/images/text_with_noise.png

In [ ]:
image_example = "text_with_noise.png"
image_example_df = spark.read.format("binaryFile").load(image_example).cache()

## Define transformers and pipeline

In [ ]:
illumination_homogenized = DocQualityClassifier() \
    .setInputCol("image") \
    .setOutputCol("quality")

pipeline = PipelineModel(stages=[
    illumination_homogenized
])

## Execution

In [ ]:
for r in pipeline.transform(image_example_df).select("image", "quality").collect():
    display_image(r.image)
    print(f"Document quality (1/bad - 5/good) = {r.quality}")